# EDA on the original Dataset
This notebook provides comprehensive EDA for the original `flights_sample_3m.csv` dataset.

## Objectives:
1. Load Dataset in from Databricks Delta Lake
2. Understand the main characteristics of the dataset
3. Uncover insights and Identify patterns in the data
4. Use Visualizations and Summary stats to gain a deeper understanding of the Data's structure



## Load Data in from Databricks Delta Lake:

In [ ]:
from pyspark.sql import SparkSession

# Initialize Spark session (Databricks usually already has one)
spark = SparkSession.builder.appName("EDA_Bronze_FlightData").getOrCreate()

# Define the Bronze table name or path
BRONZE_TABLE = "default.bronze_flights_data"  # or your actual schema.table
# Alternative if you only have the path:
# BRONZE_PATH = "/mnt/delta/bronze_flights_data"

# ----------------------------------------
# 📥 LOAD BRONZE TABLE
# ----------------------------------------
print("📥 Loading Bronze Delta table...")

# Option 1: Load by table name
df_spark = spark.read.table(BRONZE_TABLE)

# Option 2: If you have only a path
# df_spark = spark.read.format("delta").load(BRONZE_PATH)

print(f"✅ Loaded {df_spark.count():,} records")
print("📊 Schema:")
df_spark.printSchema()

# ----------------------------------------
# 🧠 OPTIONAL: CONVERT TO PANDAS FOR EDA
# ----------------------------------------
# ⚠️ Be careful with large datasets — sample first if necessary
df = df_spark.toPandas()

print("\n✅ Converted to Pandas DataFrame.")
print(df.info())
print("\nSample records:")
print(df.head())

### Below is an explanation of every column in the original dataset:

1. FL_DATE: Date of the flight.
2. AIRLINE: Name of the airline.
3. AIRLINE_DOT: DOT identifier for the airline.
4. AIRLINE_CODE: Code assigned to the airline.
5. DOT_CODE: DOT identifier.
6. FL_NUMBER: Flight number.
7. ORIGIN: Origin airport code.
8. ORIGIN_CITY: City of origin airport.
9. DEST: Destination airport code.
10. DEST_CITY: City of destination airport.
11. CRS_DEP_TIME: Scheduled departure time.
12. DEP_TIME: Actual departure time.
13. DEP_DELAY: Departure delay.
14. TAXI_OUT: Time spent taxiing out.
15. WHEELS_OFF: Time when aircraft's wheels leave the ground.
16. WHEELS_ON: Time when aircraft's wheels touch the ground.
17. TAXI_IN: Time spent taxiing in.
18. CRS_ARR_TIME: Scheduled arrival time.
19. ARR_TIME: Actual arrival time.
20. ARR_DELAY: Arrival delay.
21. CANCELLED: Indicator if the flight was cancelled (1 for cancelled, 0 for not cancelled).
22. CANCELLATION_CODE: Reason for cancellation (if applicable).
23. DIVERTED: Indicator if the flight was diverted (1 for diverted, 0 for not diverted).
24. CRS_ELAPSED_TIME: Scheduled elapsed time.
25. ELAPSED_TIME: Actual elapsed time.
26. AIR_TIME: Time spent in the air.
27. DISTANCE: Distance traveled.
28. DELAY_DUE_CARRIER: Delay due to carrier.
29. DELAY_DUE_WEATHER: Delay due to weather.
30. DELAY_DUE_NAS: Delay due to National Airspace System (NAS).
31. DELAY_DUE_SECURITY: Delay due to security.
32. DELAY_DUE_LATE_AIRCRAFT: Delay due to late aircraft arrival.
33. bronze_ingestion_timestamp: When the data was ingested into the bronze table

### EDA Setup and Data Overview: 
This cell imports the necessary libraries for data analysis and visualization, configures Seaborn plotting style, drops the bronze_ingestion_timestamp column since it is metadata, and prints a quick summary of the dataset including the number of rows and columns. Finally, it displays the first 5 rows to inspect the structure and sample values.

In [ ]:
# ==========================================
# EDA FOR BRONZE FLIGHT DATA
# ==========================================

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set(style="whitegrid", palette="Set2", font_scale=1.1)
# Drop the unnecessary Metadata Column
df = df.drop(columns=["bronze_ingestion_timestamp"], errors="ignore")
print("DATA OVERVIEW")
print("=" * 70)
print(f"Rows: {df.shape[0]:,}")
print(f"Columns: {df.shape[1]}")

display(df.head(5))

### Arrival Delay Analysis: 
This cell counts the number of flights with an arrival delay greater than 0 minutes, and calculates the percentage of delayed flights relative to the total. The results provide a quick measure of how common delays are in the dataset. From it we can see 32.6% of the flights in the dataset were delayed by at least a minute.

In [ ]:
# Ensure ARR_DELAY is numeric
df["ARR_DELAY"] = pd.to_numeric(df["ARR_DELAY"], errors="coerce")

# Count flights with arrival delay > 0
total_flights = len(df)
delayed_flights = df[df["ARR_DELAY"] > 0].shape[0]

# Calculate percentage
percent_delayed = (delayed_flights / total_flights) * 100

print(f"Total flights: {total_flights:,}")
print(f"Flights with ARR_DELAY > 0: {delayed_flights:,}")
print(f"Percentage of flights delayed: {percent_delayed:.2f}%")

### Airline Distribution: 
Visualizes the number of flights operated by each airline using a horizontal bar chart. Airlines are sorted by the total count of flights. From this chart we can see that Southwest Airlines has the most datapoints, followed by Delta and American Airlines.

In [ ]:
# ==========================================
# DISTRIBUTION OF AIRLINES
# ==========================================
# Get airlines sorted by count descendingly
df_category = df.select_dtypes(include=["object", "category"])
df_numerical = df.select_dtypes(exclude=["object"])
# Get airlines sorted by count descendingly

order_desc = df_category["AIRLINE"].value_counts().index
sns.set_context("notebook")
g = sns.countplot(y="AIRLINE", data=df_category, order=order_desc)
g.set(xlabel="Number of Flights")

### Top 20 Destination Airports: 
This chart shows the 20 airports with the highest number of arriving flights. Each bar represents the total arrivals for that airport. From it we can see that the Atlanta airport has the most arrivals in the Dataset, followed by Dallas and so on.

In [ ]:
# ==========================================
# 🌍 TOP 20 DESTINATION AIRPORTS
# ==========================================
print("\n🌍 Top 20 Destination Airports:")
top_dest = df["DEST"].value_counts().head(20)

plt.figure(figsize=(10, 6))
ax = sns.barplot(x=top_dest.values, y=top_dest.index, palette="Blues_r")

# Add count labels to each bar
for container in ax.containers:
    ax.bar_label(container, fmt="%d", label_type="edge", padding=3, fontsize=10, color="black")

plt.title("Top 20 Destination Airports")
plt.xlabel("Number of Arrivals")
plt.ylabel("Destination Airport")
plt.tight_layout()
plt.show()

### Delay Distributions (Trimmed): 
These histograms show the distribution of departure and arrival delays, limited to the range -50 to 250 minutes to reduce the impact of extreme outliers on the graphs. The KDE curve helps visualize the overall shape of the delay data. From it we can see that most flights arent delayed at all and that the distributeion of flight delays is roughly normal but is still notably skewed right. 

In [ ]:
# ==========================================
# DELAY DISTRIBUTIONS (Trimmed at 250 mins)
# ==========================================
delay_cols = ["DEP_DELAY", "ARR_DELAY"]
df_sample = df.sample(50000, random_state=42) if len(df) > 50000 else df

for col in delay_cols:
    plt.figure(figsize=(8, 4))

    # Filter out extreme outliers above 250 minutes
    data = df_sample[df_sample[col].between(-50, 250)]

    sns.histplot(data[col], bins=60, kde=True, color="steelblue")
    plt.title(f"Distribution of {col} (-50 –> 250 minutes)")
    plt.xlabel("Delay (minutes)")
    plt.xlim(-50, 250)  # explicitly bound the x-axis
    plt.tight_layout()
    plt.show()

### Monthly Average Arrival Delay by Year: 
This line plot shows how average arrival delays change month-to-month for each year in the dataset. By grouping the data by year and month, we can identify seasonal patterns and compare trends across years. We can see consistently higher flight delays in the summer months and in December, likely due to the increased air traffic during these months. We can also see significantly lower, even negative during 2020, most probably due to the Covid pandemic

In [ ]:
df["FL_DATE"] = pd.to_datetime(df["FL_DATE"], errors="coerce")
print(f"Date range: {df['FL_DATE'].min()} → {df['FL_DATE'].max()}")

# ==========================================
# MONTHLY AVERAGE DELAY BY YEAR
# ==========================================
print("\nMonthly Average Arrival Delay by Year:")

# Extract year and month for grouping
df["Year"] = df["FL_DATE"].dt.year
df["Month"] = df["FL_DATE"].dt.month

# Group by year/month
monthly_delay = df.groupby(["Year", "Month"])["ARR_DELAY"].mean().reset_index().dropna()

# Create pivot table for lineplot
plt.figure(figsize=(12, 6))

# Use a distinct color palette for clarity
years = monthly_delay["Year"].unique()
palette = sns.color_palette("tab10", n_colors=len(years))  # up to 10 distinct colors
sns.lineplot(
    data=monthly_delay,
    x="Month",
    y="ARR_DELAY",
    hue="Year",
    marker="o",
    palette=palette,
    linewidth=2.5,  # slightly thicker lines
)

plt.title("Average Monthly Arrival Delay by Year", fontsize=14)
plt.xlabel("Month", fontsize=12)
plt.ylabel("Average Arrival Delay (minutes)", fontsize=12)
plt.legend(title="Year", bbox_to_anchor=(1.05, 1), loc="upper left")  # move legend outside
plt.xticks(range(1, 13))
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### Correlation Heatmap: 
This heatmap visualizes the pairwise correlations between numeric features in the dataset. Regretably we cannot easily show the correlations between the currently catagorical data and our eventual target column ARR_DELAY before we do more preprocessing. There are several chuncks of incredibly highly correlated data in this set which makes sense because those datapoints are often intuitively related. (air time and distance for example)

In [ ]:
# ==========================================
# CORRELATION HEATMAP
# ==========================================
numeric_cols = df.select_dtypes(include=[np.number]).columns
corr = df_sample[numeric_cols].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr, cmap="coolwarm", center=0)
plt.title("Correlation Heatmap (Sample of Numeric Features)")
plt.show()